In [20]:

import os
import cv2     as cv
import numpy   as np
from ultralytics import YOLO
import calibri as cal




class Data_Collector:

    def __init__(self,imgs_dir_path, cam_calibration_path):
        self.imgs_dir_path = imgs_dir_path
        data               = np.load(cam_calibration_path,allow_pickle=True)
        self.mtx           = data['mtx']
        self.dist          = data['dist']
    
    def take_image(self, img_name , cam_idx):

        abs_path = os.path.join(self.imgs_dir_path,img_name)

        cap = cv.VideoCapture(cam_idx)
        
        if not cap.isOpened():
            print("Cannot open camera")
            exit()

        ret, frame = cap.read()

        if not ret:
            print("Can't take an image. Exiting ....")
            cap.release()
            cv.destroyAllWindows()
            return None
        else:       
            img = np.asarray(frame)
            undistored_img = cal.undistort(img,self.mtx,self.dist)
            success = cv.imwrite(abs_path,undistored_img)
            
            if success:
                print(f"Image successfully saved to {abs_path}")
               
            else:
                print(f"An error occured. The image was not able to be captured.")
                             
        cap.release()
        cv.destroyAllWindows()
        return abs_path

    
    def get_img_count(self, filepath):
        
        img_dir = os.listdir(filepath)
        
        count = 0

        for file in img_dir:
            if file.endswith(".jpg") or file.endswith(".png") or file.endswith(".jpeg"):
                count+=1

        return count

    def find_camera_idx(self):
        
        for cam_idx in range(-1,10):

            cap = cv.VideoCapture(cam_idx)
            
            if cap.isOpened():
                print(f"Camera is open on idx {cam_idx}")


        




In [21]:
obj = Data_Collector("/home/mgarcia/Pictures/","./calibrationdata.npz")

In [22]:
obj.find_camera_idx()

Camera is open on idx -1


[ WARN:0@1085.684] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1085.685] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@1085.686] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@1085.687] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@1085.688] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@1085.689] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@1085.689] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@1085.690] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@1085.690] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@1085.691] global obsensor_uvc_stream_channel.c

In [23]:
img_path = obj.take_image("img3.png",-1)

Image successfully saved to /home/mgarcia/Pictures/img3.png


In [24]:
model = YOLO("../../object_detection_model/model/mg_model_6/best.pt")

In [46]:
results = model(source = img_path,conf = .6)



image 1/1 /home/mgarcia/Pictures/img3.png: 480x640 (no detections), 246.7ms
Speed: 16.2ms preprocess, 246.7ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)


In [51]:
for r in results:
    print(r.boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([])
conf: tensor([])
data: tensor([], size=(0, 6))
id: None
is_track: False
orig_shape: (480, 640)
shape: torch.Size([0, 6])
xywh: tensor([], size=(0, 4))
xywhn: tensor([], size=(0, 4))
xyxy: tensor([], size=(0, 4))
xyxyn: tensor([], size=(0, 4))
